In [41]:
#enable autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys

sys.path.append('..')

from environments.ElevatorEnvironment import ElevatorEnvironment
from agents.llmzero import LLMTransitionModel
from agents.random_agent import RandomAgent
from agents.elevator_expert import ElevatorExpertPolicyAgent

import numpy as np
import random
import matplotlib.pyplot as plt

c:\Users\ianch\miniconda3\envs\aiplanning\Lib\site-packages\pyRDDLGym\Elevator.py:8: UserWarning: cv2 is not installed: save_as_mp4 option will be disabled.
  from pyRDDLGym.Visualizer.MovieGenerator import MovieGenerator
c:\Users\ianch\miniconda3\envs\aiplanning\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
env = ElevatorEnvironment()

c:\Users\ianch\miniconda3\envs\aiplanning\Lib\site-packages\pyRDDLGym\Examples c:\Users\ianch\miniconda3\envs\aiplanning\Lib\site-packages\pyRDDLGym\Examples\manifest.csv
Available example environment(s):
CartPole_continuous -> A simple continuous state-action MDP for the classical cart-pole system by Rich Sutton, with actions that describe the continuous force applied to the cart.
CartPole_discrete -> A simple continuous state MDP for the classical cart-pole system by Rich Sutton, with discrete actions that apply a constant force on either the left or right side of the cart.
Elevators -> The Elevator domain models evening rush hours when people from different floors in a building want to go down to the bottom floor using elevators.
HVAC -> Multi-zone and multi-heater HVAC control problem
MarsRover -> Multi Rover Navigation, where a group of agent needs to harvest mineral.
MountainCar -> A simple continuous MDP for the classical mountain car control problem.
NewLanguage -> Example with

c:\Users\ianch\miniconda3\envs\aiplanning\Lib\site-packages\pyRDDLGym\Core\Env\RDDLConstraints.py:85: UserWarning: Constraint does not have a structure of <action or state fluent> <op> <rhs>, where:
<op> is one of {<=, <, >=, >}
<rhs> is a deterministic function of non-fluents or constants only.
>> ( sum_{?f: floor} [ elevator-at-floor(?e, ?f) ] ) == 1
  warnings.warn(


In [4]:
SEED = 117
# np.random.seed(SEED)
# random.seed(SEED)

In [7]:
random_agent = RandomAgent(env, seed=SEED)
expert_agent = ElevatorExpertPolicyAgent()

In [42]:
from utils import elevator_estimate_value

In [118]:
state, _ = env.reset(seed=SEED)
history = []

In [194]:
value_ntg = elevator_estimate_value(state)
state, _, _, _, _ = env.step(1)
value_up = elevator_estimate_value(state)

print(value_ntg, value_up)

{'num-person-waiting___f0': 0, 'num-person-waiting___f1': 0, 'num-person-waiting___f2': 1, 'num-person-waiting___f3': 0, 'num-person-waiting___f4': 0, 'num-person-in-elevator___e0': 0, 'elevator-dir-up___e0': True, 'elevator-closed___e0': True, 'elevator-at-floor___e0__f0': False, 'elevator-at-floor___e0__f1': False, 'elevator-at-floor___e0__f2': True, 'elevator-at-floor___e0__f3': False, 'elevator-at-floor___e0__f4': False}
58.5
{'num-person-waiting___f0': 0, 'num-person-waiting___f1': 0, 'num-person-waiting___f2': 1, 'num-person-waiting___f3': 1, 'num-person-waiting___f4': 0, 'num-person-in-elevator___e0': 0, 'elevator-dir-up___e0': True, 'elevator-closed___e0': True, 'elevator-at-floor___e0__f0': False, 'elevator-at-floor___e0__f1': False, 'elevator-at-floor___e0__f2': False, 'elevator-at-floor___e0__f3': True, 'elevator-at-floor___e0__f4': False}
114.75
58.5 114.75


In [157]:
from agents.mcts import MCTSAgent

mcts_args = {
            "num_simulations": 1000,
            "c_puct": 100,    #should be proportional to the scale of the rewards 
            "gamma": 0.99,
            "max_depth": 40,
            "num_rollouts": 10,
            "backprop_T": 50,
        }
# agent = MCTSAgent(env, policy=llm_agent, debug=True, args=mcts_args)
agent = MCTSAgent(env, policy=None, debug=True, args=mcts_args)

state, _ = env.reset(seed=42)

In [170]:
action = agent.act(state)

 12%|█▏        | 12/100 [00:00<00:01, 57.95it/s]

-----------------------------------------
Action: 0
State:
 People waiting at floor 2: 0
People waiting at floor 3: 0
People waiting at floor 4: 0
People waiting at floor 5: 0
Elevator at floor 1.
There are 0 people in the elevator.
Elevator is moving up.
Elevator door is closed.

Rollout reward: 0
-----------------------------------------
Action 0: Q = 0.0, N = 1, prob = 0.25
-----------------------------------------
Action: 1
State:
 People waiting at floor 2: 0
People waiting at floor 3: 0
People waiting at floor 4: 0
People waiting at floor 5: 0
Elevator at floor 2.
There are 0 people in the elevator.
Elevator is moving up.
Elevator door is closed.

Rollout reward: 0
-----------------------------------------
Action 0: Q = 0.0, N = 1, prob = 0.25
Action 1: Q = 0.0, N = 1, prob = 0.25
-----------------------------------------
Action: 2
State:
 People waiting at floor 2: 0
People waiting at floor 3: 0
People waiting at floor 4: 0
People waiting at floor 5: 0
Elevator at floor 1.
There

 18%|█▊        | 18/100 [00:00<00:01, 53.22it/s]

Action 0: Q = 19.0, N = 3, prob = 0.25
Action 1: Q = 27.0, N = 3, prob = 0.25
Action 2: Q = 19.0, N = 3, prob = 0.25
Action 3: Q = 19.0, N = 3, prob = 0.25
Action 0: Q = 19.0, N = 3, prob = 0.25
Action 1: Q = 27.0, N = 3, prob = 0.25
Action 2: Q = 19.0, N = 3, prob = 0.25
Action 3: Q = 19.0, N = 3, prob = 0.25
Action 0: Q = 19.0, N = 3, prob = 0.25
Action 1: Q = 27.0, N = 3, prob = 0.25
Action 2: Q = 19.0, N = 3, prob = 0.25
Action 3: Q = 19.0, N = 3, prob = 0.25
Action 0: Q = 19.0, N = 3, prob = 0.25
Action 1: Q = 27.0, N = 3, prob = 0.25
Action 2: Q = 19.0, N = 3, prob = 0.25
Action 3: Q = 19.0, N = 3, prob = 0.25
-----------------------------------------
Action: 1
State:
 People waiting at floor 2: 0
People waiting at floor 3: 1
People waiting at floor 4: 1
People waiting at floor 5: 0
Elevator at floor 5.
There are 0 people in the elevator.
Elevator is moving up.
Elevator door is closed.

Rollout reward: 60
-----------------------------------------
Action 0: Q = 19.0, N = 3, prob =

 29%|██▉       | 29/100 [00:00<00:01, 35.95it/s]

...

Action 0: Q = 106.13793103448276, N = 29, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.13793103448276, N = 29, prob = 0.25
Action 3: Q = 105.64285714285714, N = 28, prob = 0.25
Action 0: Q = 106.13793103448276, N = 29, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.13793103448276, N = 29, prob = 0.25
Action 3: Q = 105.64285714285714, N = 28, prob = 0.25
Action 0: Q = 106.13793103448276, N = 29, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.13793103448276, N = 29, prob = 0.25
Action 3: Q = 105.64285714285714, N = 28, prob = 0.25
Action 0: Q = 106.13793103448276, N = 29, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.13793103448276, N = 29, prob = 0.25
Action 3: Q = 105.64285714285714, N = 28, prob = 0.25
Action 0: Q = 106.13793103448276, N = 29, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.13793103448276,

 99%|█████████▉| 99/100 [00:07<00:00,  9.03it/s]

Action 0: Q = 106.6, N = 30, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.13793103448276, N = 29, prob = 0.25
Action 3: Q = 106.13793103448276, N = 29, prob = 0.25
Action 0: Q = 106.6, N = 30, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.13793103448276, N = 29, prob = 0.25
Action 3: Q = 106.13793103448276, N = 29, prob = 0.25
Action 0: Q = 106.6, N = 30, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.13793103448276, N = 29, prob = 0.25
Action 3: Q = 106.13793103448276, N = 29, prob = 0.25
Action 0: Q = 106.6, N = 30, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.13793103448276, N = 29, prob = 0.25
Action 3: Q = 106.13793103448276, N = 29, prob = 0.25
Action 0: Q = 106.6, N = 30, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.13793103448276, N = 29, prob = 0.25
Action 3: Q = 106.13793103448276, N = 29, pr

100%|██████████| 100/100 [00:07<00:00, 13.02it/s]

Action 0: Q = 106.6, N = 30, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.6, N = 30, prob = 0.25
Action 3: Q = 106.6, N = 30, prob = 0.25
Action 0: Q = 106.6, N = 30, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.6, N = 30, prob = 0.25
Action 3: Q = 106.6, N = 30, prob = 0.25
Action 0: Q = 106.6, N = 30, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.6, N = 30, prob = 0.25
Action 3: Q = 106.6, N = 30, prob = 0.25
Action 0: Q = 106.6, N = 30, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.6, N = 30, prob = 0.25
Action 3: Q = 106.6, N = 30, prob = 0.25
Action 0: Q = 106.6, N = 30, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.6, N = 30, prob = 0.25
Action 3: Q = 106.6, N = 30, prob = 0.25
Action 0: Q = 106.6, N = 30, prob = 0.25
Action 1: Q = 40.333333333333336, N = 9, prob = 0.25
Action 2: Q = 106.6, N = 3

In [122]:
value = elevator_estimate_value(state)
value

18.0

In [81]:
checkpoint = env.checkpoint()

In [103]:
env.restore_checkpoint(checkpoint)

In [170]:
state, reward, done, info, _ = env.step(action)
print(env.state_to_text(state))

People waiting at floor 2: 1
People waiting at floor 3: 1
People waiting at floor 4: 1
People waiting at floor 5: 0
Elevator at floor 3.
There are 0 people in the elevator.
Elevator is moving down.
Elevator door is open.



In [171]:
action = agent.act(state)

100%|██████████| 1000/1000 [00:46<00:00, 21.34it/s]

Action 0: Q = 36.247252747252745, N = 91
Action 1: Q = 31.544776119402986, N = 67
Action 2: Q = 15.910714285714286, N = 28
Action 3: Q = 99.72972972972973, N = 814
